In [21]:
import numpy as np 
import matplotlib.pyplot as plt
import numba as nb
import cython as cy
import scipy
import const 
from scipy.special import gamma
from scipy.special import kv
from scipy import *
from scipy.integrate import odeint
import math as m
import mpmath

In [22]:
# Defining global variables 
#n0 = 0.001
n0 = 1 
epsilon_B = 0.0001
tetav_c = 0 
sh_max=100
En=10**43
mu_val = 0

In [23]:
def min(a, b):
    if a < b:
        return a
    else:
        return b


In [24]:
#Définition des fonctions de Bessel de première espèce (avec une limite finie en 0)



def K1(x) :
    return kv(1, x)

def K2(x) :
    return kv(2, x)

def K3(x) :
    return kv(3, x)

def f1(x):
    if x < 650:
        return K3(x)/K2(x)
    else :
        return 1 + 2.5/x



def f2(x):
    if x < 650:
        return K2(x)/(3*K3(x) + K1(x) - 4*K2(x))
    else: 
        return x/(3 + np.power(x,2) - 4*x)

# Defining gamma_maj function
def gamma_maj(ze): 
    return f1(ze) - 1/ze

def gamma_chapeau_prime(ze):
    return 1 + 4*f2(ze)/ze

gamma_maj =np.vectorize(gamma_maj)
gamma_chapeau_prime = np.vectorize(gamma_chapeau_prime)





In [25]:
# Depend on tetav which is a global constant
def mu(teta, phi):
    return np.sin(teta) * np.sin(tetav_c) * np.cos(phi) + np.cos(teta) * np.cos(tetav_c)


# Depend on distibution of E n0 is a global constant 
def C_BM(E):
    return np.sqrt(17 * E / (8 * np.pi * n0 * const.m_p * const.celer**5))

def C_ST(E):
    return (2/5) * 1.15 * (E / (n0 * const.m_p * const.celer**5))**(1/5)

def radius(T, teta, phi):
    C_BM_val = C_BM(En)
    C_ST_val = C_ST(En)
    mu_val = mu(teta, phi)
    def f(R, t):
        t =T+ mu_val * R / const.celer
        t_safe = np.where(t>0, t, 0.000001)
        mine = np.minimum((C_BM_val**2) * (t_safe) ** (-3) + (C_ST_val**2) * (t_safe) ** (-6 / 5), sh_max**2)
        return np.sqrt(mine / (1 + mine)) * const.celer
    
    Rp = -T*const.celer/mu_val
    Rs = 1e25
    cpt = 0
    while abs(Rp - Rs)/abs(Rs) > 10 ** (-2) and cpt < 20:
        cpt += 1
        #print(Rs)
        time1 = [0,T+mu_val*Rs/const.celer]
        sol = odeint(f,y0=[-T*const.celer/mu_val], t=time1)
        Rp = Rs
        Rs = sol[1][0]
    return Rs

# Retourne beta_sh
 
def shock_speed(E, T, teta, phi):
    # mu_val est une variable globale qui est modifiée dans le calcul de l'intégrante. 
    radius_value = radius(T, teta, phi)
    t = T + mu_val * radius_value / const.celer
    mine = min((C_BM(E)**2) * (t) ** (-3) + (C_ST(E)**2) * (t) ** (-6/5), sh_max**2)
    beta_sh = np.sqrt(mine / (1 + mine))
    return beta_sh




In [26]:
# Détermination de Ksi, car il est défini implicitement ici. 
#Find the rpoblem with ksi ici
from scipy.optimize import fsolve

def eq(gamma_sh,ze):
    a = (gamma_maj(ze) + 1)*(1 + gamma_chapeau_prime(ze)*(gamma_maj(ze) - 1))**2
    b = 2 + gamma_chapeau_prime(ze)*(2 - gamma_chapeau_prime(ze)*(gamma_maj(ze) - 1))
    return gamma_sh**2 - a/b

def sol(gamma_sh): #Sol is zeta
    def func(ze):
        return eq(gamma_sh, ze)
    solve = fsolve(func, 1)
    return solve[0]

sol_vect = np.vectorize(sol)
g_m = np.vectorize(gamma_maj)
g_c = np.vectorize(gamma_chapeau_prime)


#We can now calculate values of beta's

def beta_u(E, t, teta, phi):
    return shock_speed(E, t, teta, phi)

def beta(E, t, teta, phi):
    beta_sh = shock_speed(E, t, teta, phi)
    zeta = sol(beta_sh)
    return np.sqrt(1 - 1/(gamma_maj(zeta) ** 2))

def beta_d(E, T, teta, phi):
    beta_sh = shock_speed(E, T, teta, phi)
    beta_val = beta(E, T, teta, phi)
    return (beta_sh - beta_val) / (1 - beta_val * beta_sh)

def p(E, T, teta, phi):
    beta_u_val = beta_u(E, T, teta, phi)
    beta_d_val = beta_d(E, T, teta, phi)
    a = 3 * beta_u_val - 2 * beta_u_val * ( beta_d_val ** 2  ) + beta_d_val ** 3
    b = beta_u_val - beta_d_val
    return a/b - 2

def gamma_shf(E, n0, sh_max,t, teta, tetav, phi):
    beta_sh = shock_speed(E, n0, sh_max, t, teta, tetav, phi)
    return np.sqrt(1/(1 - beta_sh**2))

def n_prime(gamma_sh):
    return 4*gamma_maj(sol(gamma_sh))*n0

def e_i_p(gamma_sh):
    return const.m_p*((const.celer)**2) * (gamma_maj(sol(gamma_sh)) - 1)*n_prime(gamma_sh)


def gamma_prime_c( gamma_sh,t):
    return (3 * const.m_e * const.celer* gamma_maj(sol(gamma_sh)))/ (4 * const.sigma_T * epsilon_B *e_i_p( gamma_sh) * t*n0)


def gamma_prime_m( gamma_sh, p):
    arg = ((p - 2)*const.m_p/(p - 1)*const.m_e)*const.epsi_e*(gamma_maj(sol(gamma_sh)) - 1)
    return max(1, arg)

def B_prime( gamma_sh):
        return (np.sqrt(8*const.pi*epsilon_B*e_i_p(gamma_sh)))

def nu_prime_m(gamma_sh, p):
    return 3/16 * gamma_prime_m(gamma_sh, p)**2 * const.q_e*B_prime(gamma_sh) / (const.m_e * const.celer)

def nu_prime_c( t, gamma_sh):
    return 3/16 * gamma_prime_c(gamma_sh, t)**2 * const.q_e *B_prime(gamma_sh) / (const.m_e * const.celer)


In [27]:
def power_I(x,y):
    return 1/np.power(x,y)

#Rapport de comparaison pour les refroidissemts entre nu_prim_c et nu_prim_m
def Kappa(t,gamma_sh,p):
    return nu_prime_c(t,gamma_sh)/nu_prime_m(gamma_sh, p)

def n_prime_r(gamma_sh, p):
    return n_prime(gamma_sh)*min(1,((p - 2)*const.m_p/(p - 1)*const.m_e)*const.epsi_e*(gamma_maj(sol(gamma_sh)) - 1))

def FastcEmission(x,p,t,gamma_sh):
    if x > Kappa(t,gamma_sh,p):
        return np.power(Kappa(t,gamma_sh,p),-1/3)*np.power(x,1/3)
    elif x >= Kappa(t,gamma_sh,p) and x <1:
        return np.power(Kappa(t,gamma_sh,p),1/2)*power_I(x,-1/2)
    else:
         return power_I(Kappa(t,gamma_sh,p),-1/2)*power_I(x,-p/2)

#Cas du slow-cooling
def SlowcEmission(x,p,t,gamma_sh):
    if x < 1:
        return np.power(x,1/3)
    elif x>=1 and x < Kappa(t,gamma_sh,p):
        return power_I(x,-(p-1)/2)
    else:
        return power_I(Kappa(t,gamma_sh,p),-1/2)*power_I(x,-p/2)

#Fonction d'émission
def Emission_function(x,p,t,gamma_sh):
    if Kappa(t,gamma_sh,p) > 1:
        return SlowcEmission(x,p,t,gamma_sh)
    else:
        return FastcEmission(x,p,t,gamma_sh)


def fact_denorm(p,gamma_sh):
    return 0.88*(256*(p- 1)*(const.q_e**3)*n_prime_r(gamma_sh, p)
                 *B_prime(gamma_sh)/27*(3*(p - 1)*const.m_e*(const.celer**2)))



In [28]:

def p_tildt(p_val, gamma_prime_m, gamma_prime_c, beta_u_val, beta_d_val):
    if gamma_prime_m >= gamma_prime_c:
        return 2 
    else:
        return (3 * beta_u_val - 2 * beta_u_val * beta_d_val ** 2 + beta_d_val ** 3) / (beta_u_val - beta_d_val) - 2



def c_11(nu_prime, n_prime_r_val, b_prime, nu_prime_c_val, nu_prime_m_val, gamma_prime_m_val, gamma_prime_c_val, p_tildt_val):
    nu_min = min(nu_prime_m_val, nu_prime_c_val)
    gamma_min = min(gamma_prime_m_val, gamma_prime_c_val)
    a = np.power(2,6)*np.power(const.pi,5/6)*(p_tildt_val + 2)*(p_tildt_val - 1)*const.q_e*n_prime_r_val
    b = 15*(3*p_tildt_val + 2)*gamma(5/6)*np.power(gamma_min,5)*b_prime
    return (a/b)*np.power(nu_prime/nu_min, -5/3)

def c_14(nu_prime, n_prime_r_val, b_prime, nu_prime_c_val, nu_prime_m_val, gamma_prime_m_val, gamma_prime_c_val, p_tildt_val):
    nu_min = min(nu_prime_m_val, nu_prime_c_val)
    gamma_min = min(gamma_prime_m_val, gamma_prime_c_val)
    a = np.power(2, ((3 * p_tildt_val + 8) / 2))
    b = (p_tildt_val - 1) * gamma(3 / 2 + p_tildt_val / 4) * gamma(11 / 6 + p_tildt_val / 4) * gamma(1 / 6 + p_tildt_val / 4) * const.q_e * n_prime_r_val
    c = np.power(3, 3 /2) * np.power(const.pi, (p_tildt_val + 1) / 2) * gamma(2 + p_tildt_val / 4) * np.power(gamma_min, 5) * b_prime 
    return (a * b) / c * np.power(nu_prime / nu_min, -(p_tildt_val + 4) / 2)
 
def nu_prime_zero(nu_prime_c_val, nu_prime_m_val, p_tildt_val):
    nu_prime_min = min(nu_prime_m_val, nu_prime_c_val)
    a = 5*(3*p_tildt_val + 2)*gamma(3/2 + p_tildt_val/4)*gamma(11/6 + p_tildt_val/4)*gamma(1/6 + p_tildt_val/4)*gamma(5/6)
    b = (p_tildt_val + 2)*gamma(2 + p_tildt_val/4)
    c = np.power(np.power(2,3*(3*p_tildt_val - 4))/27*np.power(const.pi, 3*p_tildt_val + 8),1/(3*p_tildt_val + 2))
    return np.power(a/b,6/(3*p_tildt_val + 2))*c*nu_prime_min


def alpha_prime_nu_prime(nu_prime, p_tildt_val, nu_prime_zero_val, c_11_val, c_14_val):
    #calculs
    if nu_prime < nu_prime_zero_val:
        alpha_prime_nu_prime_zero = c_11_val
        eta =  -5/3
    else : 
        alpha_prime_nu_prime_zero = c_14_val
        eta = -(p_tildt_val + 4) / 2
    return alpha_prime_nu_prime_zero * (nu_prime / nu_prime_zero_val) ** eta

def nu_prime(nu, xi, beta, mu):
    return gamma_maj(xi) * (1 - beta * mu) * nu 




In [29]:


def alpha_nu(mu_val, beta_sh, beta_val, alpha_prime_nu_prime_val):
    zeta = sol(beta_sh)
    alpha_nu = gamma_maj(zeta)*(1-beta_val*mu_val)*alpha_prime_nu_prime_val
    return alpha_nu

def delta_s(beta_sh, R,  mu_val):
    zeta = sol(beta_sh)
    return R/((12*np.abs(mu_val - beta_sh)*(gamma_maj(zeta))**2))

def tan_nu(alpha_nu_val, delta_s_val):
    return alpha_nu_val * delta_s_val

In [30]:
nu = 20000000 #fréquence en laquelle est calculée l'intégrande

def ratio2(tau):
    if tau > 0.000001:
        return 1 - np.exp(-tau)
    else: 
        return tau



def integrand2(T,teta,phi):
    #calcul d"un premier set de variables 
    global mu_val 
    mu_val = mu(teta, phi) 
    radius_val = radius(T,teta,phi)
    tval = T + mu(teta, phi)*radius_val/const.celer
    bsh_val = shock_speed(En,T,tval,phi)
    gammash_val = np.sqrt(1/(1   -   bsh_val**2))
    zeta_val = sol(gammash_val)
    beta_val = beta(En,tval, teta,phi)
    gamma_majval = gamma_maj(zeta_val)
    nu_p_val = nu*gamma_majval*(1-beta_val*mu_val)
    p_val = p(En,tval,teta,phi)
    x_val = nu_p_val/nu_prime_m(gammash_val, p_val)
    #Calcul des premiers éléments de l'intégrande
    energ_normalized = Emission_function(x_val,p_val,tval, gammash_val)
    energ_val = energ_normalized*fact_denorm(p_val,gammash_val)
    #Premier rapport angulaire de l'intégrande
    angular_sh = np.abs(mu_val - bsh_val)*np.power(radius_val,2) / (1 - mu_val*bsh_val)*np.power(gamma_majval*(1-beta_val*mu_val),3)
    
    beta_d_val = beta_d(En, tval, teta, phi)
    beta_u_val = beta_u(En, tval, teta, phi)
    gamma_prime_c_val = gamma_prime_c(gammash_val, tval)
    gamma_prime_m_val = gamma_prime_m(gammash_val, p_val)
    nu_prime_m_val = nu_prime_m(gammash_val, p_val)
    nu_prime_c_val = nu_prime_c( tval, gammash_val)
    b_prime_val = B_prime(gammash_val)
    n_prime_r_val = n_prime_r(gammash_val, p_val)
    nu_prime_val = nu_prime(nu, zeta_val, bsh_val, mu_val) 
    p_tildt_val = p_tildt(p_val, gamma_prime_m_val, gamma_prime_c_val, beta_u_val, beta_d_val)
    nu_prime_zero_val = nu_prime_zero(nu_prime_c_val, nu_prime_m_val, p_tildt_val)
    c_11_val = c_11(nu_prime_val, n_prime_r_val, b_prime_val, nu_prime_c_val, nu_prime_m_val, gamma_prime_m_val, gamma_prime_c_val, p_tildt_val)
    c_14_val = c_14(nu_prime_val, n_prime_r_val, b_prime_val, nu_prime_c_val, nu_prime_m_val, gamma_prime_m_val, gamma_prime_c_val, p_tildt_val)
    alpha_prime_nu_prime_val = alpha_prime_nu_prime(nu_prime_val, p_tildt_val, nu_prime_zero_val, c_11_val, c_14_val)
    alpha_nu_val = alpha_nu(mu_val, bsh_val, beta_val, alpha_prime_nu_prime_val)
    delta_s_val = delta_s(bsh_val, radius_val,  mu_val)
    tau_nu_val = tan_nu(alpha_nu_val, delta_s_val)
    ratio_1 = energ_val/alpha_prime_nu_prime_val 
    ratio_2_val = ratio2(tau_nu_val)
    print(alpha_prime_nu_prime_val)
    print(ratio_1)
    print(c_11_val)
    print(c_14_val)
    print(nu_prime_val)
    print(p_tildt_val)
    print(nu_prime_zero_val)
    print(nu_prime_val, n_prime_r_val, b_prime_val, nu_prime_c_val, nu_prime_m_val, gamma_prime_m_val, gamma_prime_c_val, p_tildt_val)
    return angular_sh*ratio_1*ratio_2_val


print(integrand2(200,20*3.14/180,20*3.14/180))
    

1.1734012233e-313
inf
4.704199650577842e-85
3.2211211733096246e-199
4091514.846719903
24.093909041425764
0.029175945671776896
4091514.846719903 6.627805998113741e-57 3.5029138428184388e-06 7.91604587807063e+17 0.00038532875817460626 1 45325065444.367226 24.093909041425764
inf


c:\miniconda\envs\ue22\lib\site-packages\ipykernel_launcher.py:48: RuntimeWarning: overflow encountered in double_scalars
